In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Descomprimimos (ajusta la ruta si lo pusiste en una carpeta en Drive)
!unzip -q "/content/drive/MyDrive/dataset.zip" -d "/content/dataset"

Mounted at /content/drive


In [ ]:
import os
import shutil
import random
import yaml
from ultralytics import YOLO

# Rutas base
base_dir = '/content/dataset'
train_img_dir = os.path.join(base_dir, 'train/images')
train_lbl_dir = os.path.join(base_dir, 'train/labels')
val_img_dir = os.path.join(base_dir, 'valid/images')
val_lbl_dir = os.path.join(base_dir, 'valid/labels')

print("🚑 INICIANDO REPARACIÓN DE DATASET...")

# 1. Verificar si existe la carpeta train
if not os.path.exists(train_img_dir):
    raise FileNotFoundError(f"No encuentro la carpeta de entrenamiento en: {train_img_dir}")

# 2. Crear carpetas de validación si no existen
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# 3. Mover 20% de las imágenes para crear validación
images = [f for f in os.listdir(train_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
num_to_move = int(len(images) * 0.2) # 20%

if num_to_move > 0:
    print(f"📦 Moviendo {num_to_move} imágenes de 'train' a 'valid' para que YOLO funcione...")
    to_move = random.sample(images, num_to_move)

    for img_name in to_move:
        # Mover Imagen
        src_img = os.path.join(train_img_dir, img_name)
        dst_img = os.path.join(val_img_dir, img_name)
        shutil.move(src_img, dst_img)

        # Mover Label correspondiente (si existe)
        lbl_name = img_name.rsplit('.', 1)[0] + '.txt'
        src_lbl = os.path.join(train_lbl_dir, lbl_name)
        dst_lbl = os.path.join(val_lbl_dir, lbl_name)

        if os.path.exists(src_lbl):
            shutil.move(src_lbl, dst_lbl)
    print("✅ Validación creada exitosamente.")
else:
    print("⚠️ OJO: Ya tenías muy pocas fotos o la carpeta valid ya tenía contenido.")

# 4. Crear data.yaml CORRECTO
config = {
    'path': base_dir,
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 2,
    'names': ['botrytis_rose', 'healthy_rose']
}

yaml_path = os.path.join(base_dir, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(config, f)
print(f"📝 Configuración actualizada en: {yaml_path}")

# 5. ¡A ENTRENAR!
print("🚀 ARRANCANDO ENTRENAMIENTO (YOLOv8 Medium)...")
model = YOLO('yolov8m.pt')

model.train(
    data=yaml_path,
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    project='/content/runs/detect',
    name='train_reparado'
)

🚑 INICIANDO REPARACIÓN DE DATASET...
📦 Moviendo 36 imágenes de 'train' a 'valid' para que YOLO funcione...
✅ Validación creada exitosamente.
📝 Configuración actualizada en: /content/dataset/data.yaml
🚀 ARRANCANDO ENTRENAMIENTO (YOLOv8 Medium)...
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7869f4a76330>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804